In [118]:
import pandas as pd
import seaborn as sb
import joblib
import numpy as np

from IPython.display import display
from IPython.display import display_html 


In [2]:
data = joblib.load("../results/topmed.freeze1.1.dbvar.svafotate.jl")
data['is_single'] = data["AC"] == 1
data['gnomad'] = data["gnomAD_Count"] != 0
data['1000g'] = data["1000G_Count"] != 0
data['ccdg'] = data["CCDG_Count"] != 0
data['is_known'] = data['gnomad'] | data['1000g'] | data['ccdg']
data['is_pass'] = data['filter'].apply(lambda x: "PASS" in x)

all_known = data[data['is_pass']].groupby(['svtype', 'is_known']).size().unstack()
single_known = data[data['is_pass'] & data['is_single']].groupby(['svtype', 'is_known']).size().unstack()

all_known['known_pct'] = all_known[True] / all_known.sum(axis=1)
single_known['known_pct'] = single_known[True] / all_known.sum(axis=1)

# How many TopMed variants intersect with one of the other cohorts?

In [124]:
data['is_known'].value_counts()

False    382499
True     110086
Name: is_known, dtype: int64

In [125]:
110086 / len(data)

0.22348630185653237

# Lets look at this by SVTYPE

In [126]:
all_known.loc[["DEL", "DUP", "INV"]]

is_known,False,True,known_pct
svtype,,,
DEL,161346,80172,0.331950
DUP,77403,17856,0.187447
INV,38030,1491,0.037727


# How many TopMed singletons (AC==1) are known?

In [26]:
single_known.loc[["DEL", "DUP", "INV"]]

is_known,False,True,known_pct
svtype,,,
DEL,88623,14420,0.059706
DUP,44164,3884,0.040773
INV,22873,309,0.007819


# Intersection per-anno source
How many topmed variants are found in the known annot sources using SVAFotate and min reciprocal overlap of 80

This is just a more detailed view of the above information

In [39]:
# intersection per-anno source
rows = []
srcs = ['gnomad', '1000g', 'ccdg']
for i in srcs:
    s = data[data['is_pass'] & (data[i] == True)]['svtype'].value_counts()
    s.name = i
    rows.append(s)
peranno = pd.concat(rows, axis=1).loc[["DEL", "DUP", "INV"]]
peranno.loc["Total"] = peranno.sum()
peranno

,gnomad,1000g,ccdg
DEL,54571,34829,43119
DUP,11564,5590,8117
INV,260,398,1314
Total,66395,40817,52550


# And do it again for singletons

In [40]:
rows = []
for i in srcs:
    s = data[data['is_pass'] & (data[i] == True) & data['is_single']]['svtype'].value_counts()
    s.name = i
    rows.append(s)
peranno_single = pd.concat(rows, axis=1).loc[["DEL", "DUP", "INV"]]#, columns=["anno", "intersect"])
peranno_single.loc["Total"] = peranno.sum()
peranno_single

,gnomad,1000g,ccdg
DEL,7835,2250,7456
DUP,2060,516,1873
INV,57,57,248
Total,132790,81634,105100


# All vs all intersection
How much do each of the known annot sources intersect each other with bedtools and min reciprocal overlap of 80.

Note that these numbers are a little larger. There are fewer overlaps found by SVAFotate's 80% reciprocal overlap compared to bedtools' reciprocal overlap.

In [107]:
all_cnts = joblib.load("../results/all_known_cnts.jl")
# If we order by least to most SVs (and presumably samples?) we'll
# see the intersection pattern more clearly
order = ["1000G", "CCDG", "gnomAD", "TopMed"]
known = joblib.load("../results/known_intersect.jl")
known_single = joblib.load("../results/known_intersect_single.jl")

all_across_types = np.sum([known[_]['cnts'].loc[order][order].values for _ in known], axis=0)

m_cnts = all_cnts.loc['Total'][order]
all_across_types_pct = all_across_types.copy()
for idx, x in zip(m_cnts.index, m_cnts):
    all_across_types_pct.loc[idx] /= x

## How many variants are there in each of these cohorts?

In [122]:
all_cnts

SOURCE,CCDG,gnomAD,1000G,TopMed
SVTYPE,,,,
DEL,112328,169149,89269,241631
DUP,28962,49130,24068,95311
INV,17388,746,956,39536
Total,158678,219025,114293,376478


## For each cohort, how many of its variants intersect a call in another cohort

In [121]:
print("All intersection")
display(all_across_types.astype(int))
print("All intersection percent")
display((all_across_types_pct * 100).round(1))

All intersection


,1000G,CCDG,gnomAD,TopMed,allother
1000G,0,22630,43774,31606,59388
CCDG,23539,0,38090,43629,68550
gnomAD,44603,38100,0,55327,91574
TopMed,40858,52581,66464,0,99607


All intersection percent


,1000G,CCDG,gnomAD,TopMed,allother
1000G,0.0,19.8,38.3,27.7,52.0
CCDG,14.8,0.0,24.0,27.5,43.2
gnomAD,20.4,17.4,0.0,25.3,41.8
TopMed,10.9,14.0,17.7,0.0,26.5


## Let's break this table down by SVTYPE as well as All variants variants found only in a single sample

Note that variants in a single sample (1 observed Het or HomAlt) is a superset of singleton where AC == 1 a.k.a. 1 observed Het.

In [123]:
for svtype in known:
    known[svtype]['cnts'] = known[svtype]['cnts'].loc[order][order + ['allother']]
    known[svtype]['pcts'] = known[svtype]['pcts'].loc[order][order + ['allother']]
    known_single[svtype]['cnts'] = known_single[svtype]['cnts'].loc[order][order + ['allother']]
    known_single[svtype]['pcts'] = known_single[svtype]['pcts'].loc[order][order + ['allother']]
        
    print('\t\t\t\t\t', ('=' * 4) + svtype + ('=' * 4))
    all_cnts_styler = known[svtype]['cnts'].astype(int).style.set_table_attributes("style='display:inline'").set_caption('All Variants')
    sin_cnts_styler = known_single[svtype]['cnts'].fillna(0).astype(int).style.set_table_attributes("style='display:inline'").set_caption('Single Sample Variants')

    all_pcts_styler = known[svtype]['pcts'].style.set_table_attributes("style='display:inline'").set_caption('All Variants').format(precision=1)
    sin_pcts_styler = known_single[svtype]['pcts'].style.set_table_attributes("style='display:inline'").set_caption('Single Sample Variants').format(precision=1)

    display_html(all_cnts_styler._repr_html_()+sin_cnts_styler._repr_html_(), raw=True)
    print('\t\t\t\t      ', ('-' * 4) + 'percents' + ('-' * 4))
    display_html(all_pcts_styler._repr_html_()+sin_pcts_styler._repr_html_(), raw=True)
    print('\n')

					 ====DEL====


,1000G,CCDG,gnomAD,TopMed,allother
1000G,0,18586,35914,25916,48224
CCDG,19105,0,30816,34434,54084
gnomAD,36539,30793,0,43807,72741
TopMed,34857,43130,54622,0,80231
,1000G,CCDG,gnomAD,TopMed,allother
1000G,0,0,593,266,809
CCDG,0,0,0,0,0
gnomAD,612,0,0,1472,2014
TopMed,513,0,1997,0,2284


				       ----percents----


,1000G,CCDG,gnomAD,TopMed,allother
1000G,0.0,20.8,40.2,29.0,54.0
CCDG,17.0,0.0,27.4,30.7,48.1
gnomAD,21.6,18.2,0.0,25.9,43.0
TopMed,14.4,17.8,22.6,0.0,33.2
,1000G,CCDG,gnomAD,TopMed,allother
1000G,0.0,0.0,7.5,3.4,10.3
CCDG,nan,0.0,nan,nan,nan
gnomAD,3.1,0.0,0.0,7.4,10.1
TopMed,2.0,0.0,8.0,0.0,9.1




					 ====DUP====


...

,1000G,CCDG,gnomAD,TopMed,allother
1000G,0.0,14.8,32.1,22.5,44.1
CCDG,12.6,0.0,24.2,26.8,43.5
gnomAD,16.1,14.3,0.0,23.0,37.5
TopMed,5.9,8.5,12.2,0.0,18.8
,1000G,CCDG,gnomAD,TopMed,allother
1000G,0.0,0.0,4.0,2.5,6.4
CCDG,nan,0.0,nan,nan,nan
gnomAD,2.1,0.0,0.0,8.0,10.1
TopMed,0.5,0.0,3.0,0.0,3.5




					 ====INV====


,1000G,CCDG,gnomAD,TopMed,allother
1000G,0,481,137,276,557
CCDG,784,0,263,1424,1857
gnomAD,137,257,0,243,391
TopMed,398,1315,260,0,1492
,1000G,CCDG,gnomAD,TopMed,allother
1000G,0,0,3,2,5
CCDG,0,0,0,0,0
gnomAD,3,0,0,13,16
TopMed,2,0,13,0,15


				       ----percents----


,1000G,CCDG,gnomAD,TopMed,allother
1000G,0.0,50.3,14.3,28.9,58.3
CCDG,4.5,0.0,1.5,8.2,10.7
gnomAD,18.4,34.5,0.0,32.6,52.4
TopMed,1.0,3.3,0.7,0.0,3.8
,1000G,CCDG,gnomAD,TopMed,allother
1000G,0.0,0.0,4.1,2.7,6.8
CCDG,nan,0.0,nan,nan,nan
gnomAD,2.6,0.0,0.0,11.1,13.7
TopMed,0.0,0.0,0.2,0.0,0.3


## Interesting observation:

Even though we make more variant calls (376K compared to 2nd largest gnomad at 219K), our number of single sample calls found in other cohorts aren't proportionally larger than others. For example, 2284 of our single sample DELs intersect with any other cohort while 2014 of gnomAD's single sample DELs intersect with any other cohort.

These numbers are 410 TopMed / 435 gnomAD for Duplications and 15 TopMed /16 gnomAD for Inversions.

So TopMed has ~1.5x more variants, but approximately equal counts of single sample calls shared across cohorts.